Anki Connect Add-On needs to be installed, so Anki App (which is open) can talk to this code.
This ALlows us to adjust the TTS of each card, allowing the voice of every card!

We update each card with a new Audio Field
Then we update the Layout of each Card To include the {{Audio}} so it plays
Ultimality we create a Audio file and upload to our Anki




In [1]:
import genanki
from gtts import gTTS
import requests
import os
import json

DECK_NAME = "Ultimate Geography"
ANKI_CONNECT_URL = "http://localhost:8765"

In [2]:
def invoke(action, **params):
    """Helper to communicate with AnkiConnect"""
    requestJson = json.dumps({"action": action, "version": 6, "params": params})
    response = requests.post(ANKI_CONNECT_URL, data=requestJson).json()
    if len(response) != 2:
        raise Exception("Response has an unexpected number of fields")
    if "error" not in response:
        raise Exception("Response is missing required error field")
    if response["error"] is not None:
        raise Exception(response["error"])
    return response["result"]


# Test it by listing your decks
try:
    decks = invoke("deckNames")
    print(f"Connected! Your decks: {decks}")
except Exception as e:
    print(f"Could not connect to Anki: {e}")

Connected! Your decks: ['Deep Learning 2', 'Default', 'Total', 'Ultimate Geography']


In [3]:
# Get Model
print(f"Finding notes in deck: {DECK_NAME}...")
note_ids = invoke("findNotes", query=f'deck:"{DECK_NAME}"')
first_note = invoke("notesInfo", notes=[note_ids[0]])[0]
model_name = first_note["modelName"]
print(f"Detected Model: {model_name}")
# Ultimate Geography only has One Model

Finding notes in deck: Ultimate Geography...
Detected Model: Ultimate Geography


In [4]:
# Add Audio Fields
print("\n--- Step 1: Adding Audio Fields ---")
current_fields = invoke("modelFieldNames", modelName=model_name)
print(current_fields)
new_fields = ["Country Audio", "Capital Audio"]
for field in new_fields:
    if field not in current_fields:
        print(f"Adding field: {field}")
        invoke("modelFieldAdd", modelName=model_name, fieldName=field)
    else:
        print(f"Field '{field}' already exists. Skipping.")


--- Step 1: Adding Audio Fields ---
['Country', 'Country info', 'Capital', 'Capital info', 'Capital hint', 'Flag', 'Flag similarity', 'Map', 'Country Audio', 'Capital Audio']
Field 'Country Audio' already exists. Skipping.
Field 'Capital Audio' already exists. Skipping.


In [5]:
# Update Card Templates
print("\n--- Step 2: Updating Templates ---")
# Get current template configuration
model_templates = invoke("modelTemplates", modelName=model_name)
templates_changed = False

AUDIO_CONFIG = {
    "Country - Capital": {"front": "{{Country Audio}}", "back": "{{Capital Audio}}"},
    "Capital - Country": {"front": "{{Capital Audio}}", "back": "{{Country Audio}}"},
    # For visual cards, we usually only want audio on the answer (Back)
    "Flag - Country": {"front": None, "back": "{{Country Audio}}"},
    "Map - Country": {"front": None, "back": "{{Country Audio}}"},
}

for card_type, config in AUDIO_CONFIG.items():
    if card_type in model_templates:
        t_obj = model_templates[card_type]
        if config["front"] and config["front"] not in t_obj["Front"]:
            print(f"[{card_type}] Adding {config['front']} to FRONT")
            t_obj["Front"] += f"\n<br>{config['front']}"
            templates_changed = True

        # Update BACK (afmt)
        if config["back"] and config["back"] not in t_obj["Back"]:
            print(f"[{card_type}] Adding {config['back']} to BACK")
            t_obj["Back"] += f"\n<br>{config['back']}"
            templates_changed = True

if templates_changed:
    invoke(
        "updateModelTemplates", model={"name": model_name, "templates": model_templates}
    )
    print("Templates successfully updated!")
else:
    print("No template changes were needed.")


--- Step 2: Updating Templates ---
No template changes were needed.


In [10]:
# Generating Audio

from gtts import gTTS
import base64
from tqdm import tqdm

LANG = "en"


def text_to_base64_mp3(text):
    """Generates MP3, converts to base64, cleans up."""
    if not text or text.strip() == "":
        return None

    filename = "temp_audio_gen.mp3"
    try:
        tts = gTTS(text=text, lang="en")
        tts.save(filename)
        with open(filename, "rb") as f:
            encoded = base64.b64encode(f.read()).decode("utf-8")
        os.remove(filename)
        return encoded
    except Exception as e:
        print(f"Error generating audio for '{text}': {e}")
        return None


notes_info = invoke("notesInfo", notes=note_ids)
count = 1
total = len(notes_info)
print(total)

for note in tqdm(notes_info):
    note_id = note["noteId"]
    fields = note["fields"]

    # We will collect updates for this specific note here
    field_updates = {}

    # 1. Process Country Audio
    if "Country" in fields:
        country_text = fields["Country"]["value"]
        existing_audio = fields["Country Audio"]["value"]

        if country_text and not existing_audio:
            b64_data = text_to_base64_mp3(country_text)
            filename = f"ug_{note_id}_country.mp3"
            invoke("storeMediaFile", filename=filename, data=b64_data)
            field_updates["Country Audio"] = f"[sound:{filename}]"

    if "Capital" in fields:
        capital_text = fields["Capital"]["value"]
        existing_audio = fields["Capital Audio"]["value"]
        # Generate base64 audio

        if capital_text and not existing_audio:
            b64_data = text_to_base64_mp3(capital_text)
            if b64_data:
                # Upload to Anki
                filename = f"ug_{note_id}_capital.mp3"
                invoke("storeMediaFile", filename=filename, data=b64_data)
                # Set field value
                field_updates["Capital Audio"] = f"[sound:{filename}]"

    # print(f"[{count}/{total}] Updating note {note_id}...")
    invoke("updateNoteFields", note={"id": note_id, "fields": field_updates})

319


100%|██████████| 319/319 [31:12<00:00,  5.87s/it]
